### 004 Themenextraktion mit LDA

In [ ]:
# Standardbibliotheken
import ast

# Drittanbieter-Bibliotheken
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm # Fortschrittsanzeige für Jupyter Notebook
import gensim
from gensim import corpora
from gensim.models import CoherenceModel

# Funktion zur Berechnung der Coherence Scores für verschiedene Themenanzahlen
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    print("\nStarte Training und Coherence-Berechnung: Dies kann einen Moment dauern ...")
    coherence_scores = []
    models = []

    for num_topics in tqdm(range(start, limit + 1, step), desc="Trainiere Modelle"):
        tqdm.write(f"Trainiere Modell mit {num_topics} Themen ...")

        model = gensim.models.LdaModel(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            random_state=42,
            passes=10
        )

        coherence_model = CoherenceModel(
            model=model,
            texts=texts,
            dictionary=dictionary,
            coherence="c_v"
        )

        coherence = coherence_model.get_coherence()
        coherence_scores.append(coherence)
        models.append(model)

        tqdm.write(f"Modell mit {num_topics} Themen abgeschlossen | Coherence Score: {coherence:.4f}")

    print("\nAlle Modelle wurden erfolgreich trainiert.\n")
    return models, coherence_scores

# CSV-Datei laden (cleaned_data.csv)
data = pd.read_csv("../data/cleaned_data.csv")

# Textspalte wieder in Listenform bringen
data["lemmas"] = data["lemmas"].apply(ast.literal_eval)

print("Starte Vorbereitung für das LDA-Themenmodell (Dictionary & Korpus) ...")

# Wörterbuch und Bag-of-Words-Korpus erstellen (für gensim.models.LdaModel)
lda_dictionary = corpora.Dictionary(data["lemmas"])
lda_corpus = [lda_dictionary.doc2bow(text) for text in data["lemmas"]]

# Einfaches LDA-Modell (erste Version mit fester Themenanzahl)
lda_model = gensim.models.LdaModel(
    corpus=lda_corpus,
    id2word=lda_dictionary,
    num_topics=5, # Test
    random_state=42,
    passes=10,
    per_word_topics=True
)

# Parameter für die automatische Themenoptimierung festlegen
start, limit, step = 2, 10, 1

# Modelle mit verschiedenen Themenanzahlen trainieren und Coherence Scores berechnen
models, scores = compute_coherence_values(
    dictionary=lda_dictionary,
    corpus=lda_corpus,
    texts=data["lemmas"],
    limit=limit,
    start=start,
    step=step
)

# Ausgabe der wichtigsten Themen je Modell
print("Berechne und zeige Top-Themen je Modell:")
for i, model in enumerate(tqdm(models, desc="Top-Themen je Modell")):
    print(f"\nModell mit {i + start} Themen")
    topics = model.print_topics(num_words=5)
    for topic_id, topic in topics:
        print(f"Thema {topic_id + 1}: {topic}")

# Coherence Scores anzeigen
print("\nCoherence Scores für verschiedene Themenanzahlen:\n")
x_values = range(start, limit + 1, step)

for i in range(len(scores)):
    print(f"Themenanzahl: {x_values[i]} | Coherence Score: {scores[i]:.4f}")

# Plot: Entwicklung der Coherence Scores
plt.plot(x_values, scores, marker="o")
plt.xlabel("Anzahl der Themen")
plt.ylabel("Coherence Score")
plt.title("Coherence Score je Anzahl der Themen (LDA)")
plt.grid(True)
plt.tight_layout()
plt.show()

### Optimierung der Themenanzahl mit LDA

Die thematische Struktur wurde mit LDA analysiert und über die "Coherence Scores" optimiert.  
In einem weiteren Schritt (Notebook 5) soll zusätzlich BERTopic als alternatives Verfahren zur Anwendung kommen,  
um die thematischen Ergebnisse kontextuell zu erweitern und zu vergleichen.